In [1]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

# Моделирование растяжения плоского образца

Основные зависимости определяются законом Гука.
В общем виде для изотрпного патериала закон Гука которая выглядит следующим образом

$$
    \mathbf{\sigma} = \lambda tr(\varepsilon) \cdot \mathbf{I} + 2\mu\mathbf{\varepsilon}
$$
где $\lambda = \frac{E \nu} {(1+\nu)(1-2\nu)}$ и $\mu = \frac{E}{2(1+\nu)}$ - коэффициенты Ляме, $E$ и $\mu$ модуль Юнга и коэффициент Пуассона для рассмариваемого матераила.


Из курса сопротивления материалов вам должна быть известна немного другая запись этого же закона
$$
    \mathbf{\sigma} = {}^{4}\mathbf{E} \cdot \mathbf{\varepsilon}
$$

где $\mathbf{\sigma}$ - тензор напряжений, $\mathbf{\varepsilon}$ - тензор деформаций, ${}^{4}\mathbf{E}$ - тензор упругости.
Или же, используя нотацию Фойгта, согласно которой тензора напряжений и деформаций представляются в виде векторов содержащих 6 компонент
$$
    \mathbf{\sigma_v} = \left\{
        \begin{matrix}
            \sigma_{x}\\
            \sigma_{y}\\
            \sigma_{z}\\
            \tau_{xy}\\
            \tau_{yz}\\
            \tau_{zx}\\
        \end{matrix}
    \right\}
    \hspace{0.7cm}
    \&
    \hspace{0.7cm}
    \mathbf{\varepsilon_v} = \left\{
        \begin{matrix}
            \varepsilon_{x}\\
            \varepsilon_{y}\\
            \varepsilon_{z}\\
            \gamma_{xy}\\
            \gamma_{yz}\\
            \gamma_{zx}\\
        \end{matrix}
    \right\}

    
$$


```{warning}
Корректную запись векторов необходимо уточнить. Там есть 2 перед каким-то величинами, и порядок, кажется, другой. Подсмотреть можно в моей последней статье
```
Тогда тензор упругости примет вид
$$
    \mathbf{E} = \frac{E(1-\nu)}{(1+\nu)(1-2\nu)}\left[
        \begin{matrix}
        1 & \frac{\nu}{1-\nu} & \frac{\nu}{1-\nu} & 0 & 0 & 0 \\
        \frac{\nu}{1-\nu} & 1 & \frac{\nu}{1-\nu} & 0 & 0 & 0 \\
        \frac{\nu}{1-\nu} & \frac{\nu}{1-\nu} & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & \frac{1-2\nu}{2(1-\nu)} & 0 & 0 \\
        0 & 0 & 0 & 0 & \frac{1-2\nu}{2(1-\nu)} & 0 \\
        0 & 0 & 0 & 0 & 0 & \frac{1-2\nu}{2(1-\nu)} \\

        \end{matrix}
    \right]
$$

## Создание геометрии

![image](less2.jpg)

In [2]:
# ГОСТ 1497-84, образец плоский, тип I, образец номер №6
mm = 1e-3;
B = 40mm;
l0 = 140mm;
a0 = 20mm;
b0 = 30mm;
l = l0+2*sqrt(a0*b0);
h1 = 80mm;
r = (B-b0)/2;

In [3]:
using Gmsh
using GridapGmsh
using Gridap

gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 0);
ms_max = 1;
ms_min = 0.1;
MSFC = 3;
EL_ORDER = 1
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", MSFC);
# gmsh.option.setNumber("Mesh.MeshSizeMax", diam/2);
# gmsh.option.setNumber("Mesh.MeshSizeMin", diam/5);
gmsh.option.setNumber("Mesh.MaxNumThreads3D", 8);
gmsh.option.setNumber("Mesh.ElementOrder", EL_ORDER);
# gmsh.option.setNumber("Mesh.Algorithm3D", 9);
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 2);

gmsh.model.add("L_2_1")

Lc1 = 1mm;

factory = gmsh.model.geo;
factory.addPoint(0,             0,      0, Lc1, 1);#left-bottom corner
factory.addPoint(h1,            0,      0, Lc1, 2); #begin of first arc
factory.addPoint(h1+r,          0,      0, Lc1, 3); #center of first arc
factory.addPoint(h1+r,          r,      0, Lc1, 4); #end of first arc
factory.addPoint(h1+r+l,        r,      0, Lc1, 5); #begin of second arc
factory.addPoint(h1+r+l,        0,      0, Lc1, 6); #center of second arc
factory.addPoint(h1+r+l+r,      0,      0, Lc1, 7); #begin of second arc
factory.addPoint(h1+r+l+r+h1,   0,      0, Lc1, 8); #right-buttom corner
factory.addPoint(h1+r+l+r+h1,   B,      0, Lc1, 9); #right-top corner
factory.addPoint(h1+r+l+r,      B,      0, Lc1, 10); #begin of third arc
factory.addPoint(h1+r+l,        B,      0, Lc1, 11); #center of third arc
factory.addPoint(h1+r+l,        B-r,    0, Lc1, 12); #end of third arc
factory.addPoint(h1+r,          B-r,    0, Lc1, 13); #begin of fourth arc
factory.addPoint(h1+r,          B,      0, Lc1, 14); #center of fourth arc
factory.addPoint(h1,            B,      0, Lc1, 15); #end of fourth arc
factory.addPoint(0,             B,      0, Lc1, 16); #end of fourth arc


factory.addLine(1, 2, 1);
factory.addCircleArc(2, 3, 4, 2);

factory.addLine(4, 5, 3);
factory.addCircleArc(5, 6, 7, 4);

factory.addLine(7, 8, 5);
factory.addLine(8, 9, 6);
factory.addLine(9, 10, 7);
factory.addCircleArc(10, 11, 12, 8);

factory.addLine(12, 13, 9);
factory.addCircleArc(13, 14, 15, 10);
factory.addLine(15, 16, 11);
factory.addLine(16, 1, 12);

factory.addPhysicalGroup(1, [12], -1,"left");
factory.addPhysicalGroup(1, [6], -1, "right");
factory.addPhysicalGroup(1, [1, 2, 3, 4, 5, 7, 8, 9, 10, 11], -1, "free");

factory.synchronize();

factory.addCurveLoop(collect(1:12), 13);



factory.addPlaneSurface([13], 6)


factory.synchronize()

factory.addPhysicalGroup(2, [6], -1, "test_sample");

factory.synchronize()

gmsh.model.mesh.generate(2)


name = "test_sample.msh" |> tcf;
print(name)
gmsh.write(name)

# gmsh.fltk.run()

gmsh.finalize()


c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\test_sample.msh

In [4]:

model = GmshDiscreteModel(name);

vtk_file = "model_speciment" |> tcf;
writevtk(model, vtk_file);

Info    : Reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\test_sample.msh'...
Info    : 29 entities
Info    : 15091 nodes
Info    : 30180 elements
Info    : Done reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\test_sample.msh'


In [5]:
const E = 2.1e11
const ν = 0.3

const λ = E*ν/(1+ν)/(1-2*ν)
const μ = E/2/(1+ν)




# function σ(eps)
#     # λ = E*ν/(1+ν)/(1-2*ν)
#     # μ = E/2/(1+ν)
#     # return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)
#     return λ*tr(eps)*I(2) + 2*μ*eps
# end

σ(e) = λ*tr(e)*one(e) + 2*μ*e
# Model



σ (generic function with 1 method)

In [14]:

model = GmshDiscreteModel(name);

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},1)
V = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags = ["left", "right"])

g0 = VectorValue(0.0,  0.0)
g1 = VectorValue(1.0mm, 0.0)

U = TrialFESpace(V, [g0,g1])


Info    : Reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\test_sample.msh'...
Info    : 29 entities
Info    : 15091 nodes
Info    : 30180 elements
Info    : Done reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_2\test_sample.msh'


TrialFESpace()

In [15]:
# weak form 

# f = VectorValue(0.0, - gravity * ρ)
A(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)) )*dΩ
L(v) = 0; #∫(dot(f, v))*dΩ

op = AffineFEOperator(A, L, U,V)

x0 = zeros(Float64, num_free_dofs(V))
uh_lin = FEFunction(U,x0)
ls = BackslashSolver()
solver = LinearFESolver(ls)

uh_lin, _ = solve!(uh_lin,solver,op)

(SingleFieldFEFunction(), BackslashNumericalSetup())

In [16]:
res_file = "results_new" |> tcf;

writevtk(Ω, res_file ,  cellfields=["uh"=>uh_lin,"sigma"=>σ∘ε(uh_lin)])

(["c:\\Users\\kutsj\\OneDrive\\Education\\Programming\\Julia\\CCM_tut\\lesson_2\\results_new.vtu"],)